<a href="https://colab.research.google.com/github/neemasingh/neema/blob/master/stock_price_prediction_using_GRU/ATSA_SVM_SBI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pandas import read_csv
from pandas import datetime
from pandas import DataFrame
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot as plt
from pandas.plotting import autocorrelation_plot
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
import warnings
warnings.filterwarnings("ignore")

In [0]:
series = read_csv('/content/ICICIBANK.NS.csv')
print (series)

          Close
0    159.945007
1    161.464005
2    162.699997
3    161.164001
4    158.899994
..          ...
234  200.800003
235  192.417999
236  203.108994
237  203.709000
238  203.544998

[239 rows x 1 columns]


In [0]:
from matplotlib import pyplot
series = np.array(series)
series = series[0:50]
epsilon = 0.2 # if the difference between predicted value and true value>epsilon in ARIMA, feed into SVR
outlier = list()

size = 20
train = series[0:size]
test =  series[size:len(series)]
true_data = list()
history = list()
predictions = list()
start_point = 10
for i in range(0,start_point):
    history.append(train[i])

for t in range(start_point,len(train)):
    model = ARIMA(history, order=(0,1,0)) # arima order means that it is a random walk
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    if abs(yhat-train[t]) > epsilon :
       outlier.append(train[t]) #if a point is outlier, put its index into the set
               

history = list()
history.append(outlier[0])

sum = 0

for t in range(1, len(outlier)):
    predicted_data = np.arange(len(history),len(history)+1)
    predicted_data = np.expand_dims(predicted_data,axis=1)
    train_data = np.arange(0,len(history))
    train_data = np.expand_dims(train_data,axis=1)
    svr = SVR(kernel='linear', C=1e3, gamma = 1/1250)
    yhat = svr.fit(train_data,history).predict(predicted_data)
    true_data.append(outlier[t]) # replace train data into true data
    percentage = 1 - np.sqrt(np.mean(np.square((true_data - yhat) / true_data)))
    sum = sum + percentage

print("Accuracy : ", (sum/len(outlier))*100, " %")



Accuracy :  87.28705564740635  %
